In [1]:
import networkx as nx
import pandas as pd
import numpy as np  


def get_price(dist):
    return np.random.normal(0.20, 0.07) * dist

def add_edge(graph, row):
    departure_time = {'YEAR': row['YEAR'], 'MONTH': row['MONTH'], 'DAY': row['DAY'], 'TIME': row['SCHEDULED_DEPARTURE']}
    arrival_time = {'YEAR': row['YEAR'], 'MONTH': row['MONTH'], 'DAY': row['DAY'], 'TIME': row['SCHEDULED_ARRIVAL']}
    graph.add_edge(row['ORIGIN_AIRPORT'], row['DESTINATION_AIRPORT'], departure_time=departure_time, arrival_time=arrival_time, flight_time=row['SCHEDULED_TIME'], price=row['price'])

COLS = ['YEAR', 'MONTH', 'DAY', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DISTANCE', 'SCHEDULED_TIME', 'SCHEDULED_ARRIVAL']
N = 100000 #max = 5819079

flights = pd.read_csv("Flights-BO/data/flights.csv")[COLS]
flights = flights.head(N)
flights = flights[flights['SCHEDULED_TIME'].notna()]
flights['PRICE'] = flights['DISTANCE'].apply(lambda x: get_price(x))
print(flights.head(5))

MG = nx.MultiGraph()

for idx, row in flights.iterrows():
    departure_time = {'YEAR': row['YEAR'], 'MONTH': row['MONTH'], 'DAY': row['DAY'], 'TIME': row['SCHEDULED_DEPARTURE']}
    arrival_time = {'YEAR': row['YEAR'], 'MONTH': row['MONTH'], 'DAY': row['DAY'], 'TIME': row['SCHEDULED_ARRIVAL']}
    # print(row['ORIGIN_AIRPORT'], row['DESTINATION_AIRPORT'], departure_time, arrival_time, row['SCHEDULED_TIME'], row['PRICE'])
    MG.add_edge(row['ORIGIN_AIRPORT'], row['DESTINATION_AIRPORT'], departure_time=departure_time, arrival_time=arrival_time, flight_time=row['SCHEDULED_TIME'], price=row['PRICE'])

C:\Users\Robert\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


   YEAR  MONTH  DAY ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  \
0  2015      1    1            ANC                 SEA                    5   
1  2015      1    1            LAX                 PBI                   10   
2  2015      1    1            SFO                 CLT                   20   
3  2015      1    1            LAX                 MIA                   20   
4  2015      1    1            SEA                 ANC                   25   

   DISTANCE  SCHEDULED_TIME  SCHEDULED_ARRIVAL       PRICE  
0      1448           205.0                430  248.275527  
1      2330           280.0                750  268.136599  
2      2296           286.0                806  352.760791  
3      2342           285.0                805  457.329843  
4      1448           235.0                320  142.766391  


In [2]:
import queue
import datetime
import time
import random

In [3]:
def parse_date(date):
    if str(type(date)) == '<class \'datetime.datetime\'>':
        return date
    return datetime.datetime(date['YEAR'], date['MONTH'], date['DAY'], int('{:04d}'.format(date['TIME'])[:2]), int('{:04d}'.format(date['TIME'])[2:]))

for airport_1 in MG.nodes:  
    for airport_2 in MG.adj[airport_1]:
        if airport_1 != airport_2:
            for flight in MG.adj[airport_1][airport_2]:
                MG.adj[airport_1][airport_2][flight]['departure_time'] = parse_date(MG.adj[airport_1][airport_2][flight]['departure_time'])
                MG.adj[airport_1][airport_2][flight]['arrival_time'] = parse_date(MG.adj[airport_1][airport_2][flight]['arrival_time'])
                MG.adj[airport_1][airport_2][flight]['pheromone_level'] = 0
                MG.adj[airport_1][airport_2][flight]['pheromone_update_time'] = 0

In [4]:
# print(len(MG.nodes))

# for airport_name in MG.nodes:
#     print(airport_name, len(MG.adj['LAS'][airport_name]) if airport_name in MG.adj['LAS'] else 0)
    
    
# for flight in MG.adj['LAS']:
#     print(MG.adj['LAS'][flight])
#     print('\n\n\n')
    
# print(MG.adj['LAS']['HNL'])    
    
# for i in range(0, len(MG.adj['LAS']['HNL'])):
#     print(MG.adj['LAS']['HNL'][i-1]['departure_time'])

In [5]:
ANTS_NUMBER = 1000
ANTS_SPAWN_ITERS = 10
SIMULATION_TIME_S = 20
SIMULATION_ITERS_NUM = 100000

In [14]:
class Ant:
    def __init__(self, curr_time, curr_airport):
        self.curr_time = curr_time
        self.curr_airport = curr_airport
        self.birth_time = curr_time
        
        self.curr_trav_cost = 0
        self.curr_conn_numb = 0
        self.mode = 0
        self.path = [curr_airport]
        
    def __lt__ (self, other):
        if self.mode != other.mode:
            return self.mode < other.mode
        return self.curr_time < other.curr_time
        
    def __gt__ (self, other):
        if self.mode != other.mode:
            return self.mode > other.mode
        return self.curr_time > other.curr_time

    def __eq__ (self, other):
        return self.mode == other.mode and self.curr_time == other.curr_time

    def __ne__ (self, other):
        return not self.__eq__(other)
    
    def _print_ant(self):
        print(f'\ntravel cost {self.curr_trav_cost}\nconnections number {self.curr_conn_numb}\nflights {self.path}\n')
        
    def _update(self, next_flight):
        self.curr_time = next_flight[2]['arrival_time']
        self.curr_airport = next_flight[0]
        self.curr_trav_cost += next_flight[2]['price']
        self.curr_conn_numb += 1
        self.path.append((next_flight[0], next_flight[2]['departure_time'], next_flight[2]['arrival_time']))

In [16]:
class AntColonyAlgorithm:
    def __init__(self, flights, origin, destination, min_time, max_time, min_conn_time, max_conn_numb, max_price):
        self.flights = flights
        self.origin = origin
        self.destionation = destination
        self.min_time = min_time
        self.max_time = max_time
        self.min_conn_time = min_conn_time
        self.max_conn_numb = max_conn_numb
        self.max_price = max_price

        self.global_time = 0
        self.events = queue.PriorityQueue()

    def _init_ants(self):
        time_available_min = (self.max_time - self.min_time).total_seconds() // 60

        ants_spawn_gap = time_available_min // ANTS_SPAWN_ITERS
        for i in range(ANTS_SPAWN_ITERS):
            for j in range(ANTS_NUMBER // ANTS_SPAWN_ITERS):
                self.events.put((i * ants_spawn_gap, Ant(self.min_time + datetime.timedelta(minutes=i * ants_spawn_gap), self.origin)))
                
    def _run_next_event(self):
        curr_time, curr_ant = self.events.get()[0], self.events.get()[1]
        available_flights = self._find_available_flights(curr_time, curr_ant)
        next_flight = self._choose_flight(available_flights, curr_ant)
        if next_flight == None:
            self._kill_ant(curr_ant)
        else:
            time_diff = (next_flight[2]['arrival_time'] - curr_ant.curr_time).total_seconds() // 60
            new_time = curr_time + time_diff
            
            curr_ant._update(next_flight) 
            if next_flight[0] == self.destionation:
                curr_ant._print_ant()
            else:
                self.events.put((new_time, curr_ant))
        
    def _find_available_flights(self, curr_time, curr_ant): #change to binary search
        all_flights = self.flights[curr_ant.curr_airport]
        available_flights = []
        for airport in all_flights:
            for flight_idx in range(len(self.flights.adj[curr_ant.curr_airport][airport])):
                flight = self.flights.adj[curr_ant.curr_airport][airport][flight_idx]
                if curr_ant.curr_time + datetime.timedelta(minutes=self.min_conn_time) < flight['departure_time']:
                    available_flights.append((airport, flight_idx, flight))
                    break
        return available_flights
    
    def _choose_flight(self, available_flights, curr_ant):
        available_flights_copy = []
        for (airport, index, flight) in available_flights:
            if airport == self.destionation:
                return (airport, index, flight)
            if flight['arrival_time'] < self.max_time:
                available_flights_copy.append((airport, index, flight))
    
        if len(available_flights_copy) == 0:
            return None
        return available_flights_copy[random.randint(0, len(available_flights_copy)-1)]
                
    def _kill_ant(self, curr_ant): #todo - spawning new ant
        print(f'ant dies :c', curr_ant._print_ant())

    def run(self):
        self._init_ants()
        
#         while not self.events.empty():
#             print(self.events.get())  
        
#         start_time = time.time()
#         counter = 0
#         while not self.events.empty() and (counter % 100 != 0 or time.time() - start_time < SIMULATION_TIME_S):
#             self._run_next_event()
#             counter += 1
            
        counter = 0
        while not self.events.empty() and counter < SIMULATION_ITERS_NUM:
            self._run_next_event()
            counter += 1
    
            
        



simulation = AntColonyAlgorithm(MG, 'LAS', 'JAX', datetime.datetime(2015, 1, 1, 8, 0, 0), datetime.datetime(2015, 1, 8, 8, 0, 0), 90, 5, 5000)
simulation.run()



travel cost 710.2334590125311
connections number 2
flights ['LAS', ('BWI', datetime.datetime(2015, 1, 1, 10, 0), datetime.datetime(2015, 1, 1, 12, 30)), ('JAX', datetime.datetime(2015, 1, 1, 15, 50), datetime.datetime(2015, 1, 1, 17, 55))]


travel cost 655.7799408932322
connections number 2
flights ['LAS', ('DTW', datetime.datetime(2015, 1, 1, 11, 17), datetime.datetime(2015, 1, 1, 12, 46)), ('JAX', datetime.datetime(2015, 1, 3, 10, 45), datetime.datetime(2015, 1, 3, 13, 10))]


travel cost 661.3657323329027
connections number 2
flights ['LAS', ('MSP', datetime.datetime(2015, 1, 1, 14, 55), datetime.datetime(2015, 1, 1, 16, 18)), ('JAX', datetime.datetime(2015, 1, 2, 8, 10), datetime.datetime(2015, 1, 2, 10, 23))]


travel cost 417.8855542707707
connections number 3
flights ['LAS', ('AMA', datetime.datetime(2015, 1, 1, 13, 45), datetime.datetime(2015, 1, 1, 13, 55)), ('IAH', datetime.datetime(2015, 1, 1, 16, 43), datetime.datetime(2015, 1, 1, 18, 20)), ('JAX', datetime.datetime(2015,



travel cost 446.00851502737396
connections number 2
flights ['LAS', ('IAH', datetime.datetime(2015, 1, 4, 7, 0), datetime.datetime(2015, 1, 4, 8, 20)), ('JAX', datetime.datetime(2015, 1, 4, 10, 16), datetime.datetime(2015, 1, 4, 13, 19))]


travel cost 432.4499462606908
connections number 2
flights ['LAS', ('IAD', datetime.datetime(2015, 1, 4, 8, 5), datetime.datetime(2015, 1, 4, 10, 25)), ('JAX', datetime.datetime(2015, 1, 4, 12, 35), datetime.datetime(2015, 1, 4, 14, 43))]


travel cost 619.582614381854
connections number 3
flights ['LAS', ('CVG', datetime.datetime(2015, 1, 3, 20, 10), datetime.datetime(2015, 1, 3, 21, 35)), ('MSP', datetime.datetime(2015, 1, 4, 7, 45), datetime.datetime(2015, 1, 4, 10, 53)), ('JAX', datetime.datetime(2015, 1, 4, 19, 40), datetime.datetime(2015, 1, 4, 23, 36))]


travel cost 441.48346396707507
connections number 3
flights ['LAS', ('SAT', datetime.datetime(2015, 1, 4, 7, 0), datetime.datetime(2015, 1, 4, 8, 0)), ('ATL', datetime.datetime(2015, 1, 4,


travel cost 1310.077333602866
connections number 7
flights ['LAS', ('GEG', datetime.datetime(2015, 1, 5, 9, 20), datetime.datetime(2015, 1, 5, 11, 45)), ('PHX', datetime.datetime(2015, 1, 5, 13, 45), datetime.datetime(2015, 1, 5, 15, 30)), ('RNO', datetime.datetime(2015, 1, 5, 17, 50), datetime.datetime(2015, 1, 5, 18, 40)), ('SFO', datetime.datetime(2015, 1, 5, 20, 18), datetime.datetime(2015, 1, 5, 21, 28)), ('PHX', datetime.datetime(2015, 1, 6, 6, 0), datetime.datetime(2015, 1, 6, 8, 55)), ('IND', datetime.datetime(2015, 1, 6, 10, 50), datetime.datetime(2015, 1, 6, 13, 5)), ('SFO', datetime.datetime(2015, 1, 7, 7, 45), datetime.datetime(2015, 1, 7, 9, 57))]

ant dies :c None

travel cost 476.5410307308098
connections number 3
flights ['LAS', ('AMA', datetime.datetime(2015, 1, 6, 10, 55), datetime.datetime(2015, 1, 6, 14, 50)), ('IAH', datetime.datetime(2015, 1, 6, 16, 47), datetime.datetime(2015, 1, 6, 18, 26)), ('JAX', datetime.datetime(2015, 1, 7, 6, 30), datetime.datetime(2015, 

In [ ]:
print(datetime.datetime(2015, 1, 1, 8, 0) + datetime.timedelta(seconds=10))